In [2]:
import pandas as pd
import numpy as np
import os
import csv

#------------------------------------------------------------
# ファイルパスの設定 - ここを変更してください
#------------------------------------------------------------
# データファイルのパス
path = "G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/data"
file_name = "Tshizuoka2023_0404_50features.csv"  # この部分を分析したいファイル名に変更
file_path = f"{path}/{file_name}"

# 結果の出力先
output_path = "G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/result"
#------------------------------------------------------------

def detect_file_delimiter(file_path):
    """
    ファイルの区切り文字を検出する関数
    
    Parameters:
    file_path (str): ファイルパス
    
    Returns:
    str: 検出された区切り文字 ('\t', ',', ';' など)
    """
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        first_line = f.readline().strip()
        
        # タブが含まれていればタブ区切り
        if '\t' in first_line:
            return '\t'
        # カンマが含まれていればカンマ区切り
        elif ',' in first_line:
            return ','
        # セミコロンが含まれていればセミコロン区切り
        elif ';' in first_line:
            return ';'
        # それ以外の場合はカンマを返す
        else:
            return ','

def calculate_thresholds_and_export_csv(df, output_dir, a_values=[1.0,1.5,2.0]):
    """
    MOCAスコアデータの閾値計算と各区分の人数集計を行い、結果をCSVに出力する関数
    
    Parameters:
    df (DataFrame): 年齢とMOCAスコアを含むデータフレーム
    output_dir (str): 出力ディレクトリパス
    a_values (list): 標準偏差の倍率（デフォルト: [0.5, 1.0, 1.5]）
    
    Returns:
    dict: 各a値に対する集計結果のデータフレーム
    """
    # 出力ディレクトリの確認
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 年齢グループ列がなければ追加
    if 'AgeGroup' not in df.columns:
        df['AgeGroup'] = df['Age'].apply(lambda age: assign_age_group(age))
    
    # 年齢グループごとの統計量を計算
    group_stats = df.groupby('AgeGroup')['MoCA'].agg(['count', 'mean', 'std']).reset_index()
    
    # 結果を格納する辞書
    results = {}
    
    for a in a_values:
        # 各年齢グループの閾値を計算
        thresholds = {}
        for _, row in group_stats.iterrows():
            age_group = row['AgeGroup']
            mean_val = row['mean']
            std_val = row['std']
            lower_threshold = mean_val - a * std_val
            upper_threshold = mean_val + a * std_val
            
            # 上限閾値が26未満の場合は26を使用する
            effective_upper = max(upper_threshold, 26.0)
            
            thresholds[age_group] = {
                'mean': mean_val,
                'std': std_val,
                'lower': lower_threshold,
                'upper': upper_threshold,
                'effective_upper': effective_upper  # 有効上限閾値を追加
            }
        
        # Target列の作成
        column_name = f'Target_a{a}'
        
        def assign_target(row):
            age_group = row['AgeGroup']
            moca_score = row['MoCA']
            lower_threshold = thresholds[age_group]['lower']
            effective_upper = thresholds[age_group]['effective_upper']  # 有効上限閾値を使用
            
            # 条件1: MOCAが下限閾値未満ならTarget=1
            if moca_score < lower_threshold:
                return 1
            
            # 条件2: MOCAが有効上限閾値以上ならTarget=0
            if moca_score >= effective_upper:
                return 0
            
            # それ以外はTarget=2
            return 2
        
        df[column_name] = df.apply(assign_target, axis=1)
        
        # 各年齢グループごとの統計情報を集計
        summary_data = []
        
        for age_group in sorted(df['AgeGroup'].unique()):
            group_data = df[df['AgeGroup'] == age_group]
            
            # 閾値情報
            threshold = thresholds[age_group]
            mean_val = threshold['mean']
            std_val = threshold['std']
            lower_threshold = threshold['lower']
            upper_threshold = threshold['upper']
            effective_upper = threshold['effective_upper']  # 有効上限閾値
            
            # 各区分の人数
            target0_count = (group_data[column_name] == 0).sum()
            target1_count = (group_data[column_name] == 1).sum()
            target2_count = (group_data[column_name] == 2).sum()
            
            # 下限未満と上限以上の人数
            below_lower_count = (group_data['MoCA'] < lower_threshold).sum()
            above_upper_count = (group_data['MoCA'] >= upper_threshold).sum()
            above_effective_upper_count = (group_data['MoCA'] >= effective_upper).sum()  # 有効上限以上の人数
            
            # 集計データの追加
            summary_data.append({
                'AgeGroup': age_group,
                'SampleSize': len(group_data),
                'MeanMOCA': mean_val,
                'StdDevMOCA': std_val,
                'LowerThreshold': lower_threshold,
                'BelowLowerCount': below_lower_count,
                'UpperThreshold': upper_threshold,
                'AboveUpperCount': above_upper_count,
                'EffectiveUpperThreshold': effective_upper,  # 有効上限閾値
                'AboveEffectiveUpperCount': above_effective_upper_count,  # 有効上限以上の人数
                'Target0Count': target0_count,
                'Target1Count': target1_count,
                'Target2Count': target2_count,
                'Target0Percent': target0_count / len(group_data) * 100,
                'Target1Percent': target1_count / len(group_data) * 100,
                'Target2Percent': target2_count / len(group_data) * 100
            })
        
        # 結果をデータフレームに変換
        summary_df = pd.DataFrame(summary_data)
        
        # Target0とTarget1の合計
        total_target0 = summary_df['Target0Count'].sum()
        total_target1 = summary_df['Target1Count'].sum()
        
        # 結果を格納
        results[a] = {
            'summary': summary_df,
            'total_target0': total_target0,
            'total_target1': total_target1
        }
        
        # 詳細な集計結果をCSVに出力
        detailed_output_file = os.path.join(output_dir, f'moca_detailed_a{a}.csv')
        summary_df.to_csv(detailed_output_file, index=False)
        print(f"詳細な集計結果をCSVに出力しました: {detailed_output_file}")
        
        # シンプルな形式での出力（0:1の比率と年齢群ごとの閾値・人数）
        simple_data = []
        simple_data.append(['Target0:Target1', f'{total_target0}:{total_target1}'])
        simple_data.append([])  # 空行
        simple_data.append(['AgeGroup', 'LowerThreshold', 'BelowLowerCount', 'UpperThreshold', 'AboveUpperCount', 'EffectiveUpperThreshold', 'AboveEffectiveUpperCount'])
        
        for _, row in summary_df.iterrows():
            simple_data.append([
                row['AgeGroup'],
                f"{row['LowerThreshold']:.2f}",
                str(int(row['BelowLowerCount'])),
                f"{row['UpperThreshold']:.2f}",
                str(int(row['AboveUpperCount'])),
                f"{row['EffectiveUpperThreshold']:.2f}",  # 有効上限閾値
                str(int(row['AboveEffectiveUpperCount']))  # 有効上限以上の人数
            ])
        
        simple_output_file = os.path.join(output_dir, f'moca_simple_a{a}.csv')
        with open(simple_output_file, 'w', newline='') as f:
            for row in simple_data:
                f.write(','.join(row) + '\n')
        print(f"シンプルな集計結果をCSVに出力しました: {simple_output_file}")
    
    # すべての結果をまとめた形式
    combined_data = []
    combined_data.append(['a_value', 'Target0:Target1'])
    
    for a in a_values:
        combined_data.append([f'{a}', f'{results[a]["total_target0"]}:{results[a]["total_target1"]}'])
    
    combined_data.append([])  # 空行
    combined_data.append(['a_value', 'AgeGroup', 'LowerThreshold', 'BelowLowerCount', 'UpperThreshold', 'AboveUpperCount', 'EffectiveUpperThreshold', 'AboveEffectiveUpperCount'])
    
    for a in a_values:
        for _, row in results[a]['summary'].iterrows():
            combined_data.append([
                f'{a}',
                row['AgeGroup'],
                f"{row['LowerThreshold']:.2f}",
                str(int(row['BelowLowerCount'])),
                f"{row['UpperThreshold']:.2f}",
                str(int(row['AboveUpperCount'])),
                f"{row['EffectiveUpperThreshold']:.2f}",  # 有効上限閾値
                str(int(row['AboveEffectiveUpperCount']))  # 有効上限以上の人数
            ])
    
    combined_output_file = os.path.join(output_dir, 'moca_combined_results.csv')
    with open(combined_output_file, 'w', newline='') as f:
        for row in combined_data:
            f.write(','.join(row) + '\n')
    print(f"すべての結果をまとめたCSVを出力しました: {combined_output_file}")
    
    return results

def assign_age_group(age):
    """年齢を年齢グループに分類"""
    if age <= 69:
        return "~69"
    elif 70 <= age <= 74:
        return "70~74"
    elif 75 <= age <= 79:
        return "75~79"
    else:
        return "80~"

def clean_and_prepare_data(file_path):
    """
    ファイルを読み込み、データの前処理を行う関数
    
    Parameters:
    file_path (str): 入力ファイルパス
    
    Returns:
    DataFrame: 前処理済みのデータフレーム
    """
    try:
        # ファイルの区切り文字を検出
        delimiter = detect_file_delimiter(file_path)
        print(f"検出された区切り文字: '{delimiter}'")
        
        # ファイルを読み込む
        df = pd.read_csv(file_path, delimiter=delimiter)
        print(f"元のカラム: {df.columns.tolist()}")
        
        # カンマ区切りの列名を修正（'InspectionDateAndId,MoCA,Age'のような問題に対処）
        if len(df.columns) == 1 and ',' in df.columns[0]:
            # 列名を分割して内容を抽出
            first_row = df.iloc[0].values[0].split(',')
            column_names = df.columns[0].split(',')
            
            # 新しいデータフレームを作成
            new_data = [r.split(',') for r in df.iloc[:, 0].values]
            df = pd.DataFrame(new_data, columns=column_names)
            print(f"カラムを修正しました: {df.columns.tolist()}")
        
        # MOCAとAgeのカラムを探す
        moca_column = None
        age_column = None
        
        for col in df.columns:
            col_lower = col.lower()
            if 'moca' in col_lower or 'モカ' in col_lower:
                moca_column = col
            elif 'age' in col_lower or '年齢' in col_lower:
                age_column = col
        
        # 必要なカラムが見つからない場合
        if not moca_column or not age_column:
            # カラムが2つだけならMOCAとAgeと仮定
            if len(df.columns) == 2:
                df.columns = ['MoCA', 'Age']
                moca_column = 'MoCA'
                age_column = 'Age'
            else:
                # カラムの内容を調べて推測
                for col in df.columns:
                    # 数値のみのカラムを探す
                    try:
                        values = pd.to_numeric(df[col], errors='coerce')
                        non_na_values = values.dropna()
                        
                        # MOCAスコアの特徴: 0-30の範囲
                        if non_na_values.min() >= 0 and non_na_values.max() <= 30 and not moca_column:
                            moca_column = col
                            print(f"MOCAスコアと推測されるカラム: {col}")
                        # 年齢の特徴: 通常50-100の範囲
                        elif non_na_values.min() >= 50 and non_na_values.max() <= 100 and not age_column:
                            age_column = col
                            print(f"年齢と推測されるカラム: {col}")
                    except:
                        continue
        
        # 必要なカラムが見つからない場合はエラー
        if not moca_column or not age_column:
            raise ValueError(f"MOCAスコアと年齢を特定できませんでした。カラム: {df.columns.tolist()}")
        
        # 必要なカラムだけ抽出して名前を標準化
        df_clean = df[[moca_column, age_column]].copy()
        df_clean.columns = ['MoCA', 'Age']
        
        # 数値に変換
        df_clean['MoCA'] = pd.to_numeric(df_clean['MoCA'], errors='coerce')
        df_clean['Age'] = pd.to_numeric(df_clean['Age'], errors='coerce')
        
        # 無効な値を削除
        orig_len = len(df_clean)
        df_clean = df_clean.dropna()
        if len(df_clean) < orig_len:
            print(f"{orig_len - len(df_clean)}行の無効なデータを削除しました")
        
        return df_clean
        
    except Exception as e:
        print(f"データ読み込みエラー: {e}")
        import traceback
        traceback.print_exc()
        return None

# メイン処理
if __name__ == "__main__":
    try:
        # ファイルパスの確認
        if not os.path.exists(file_path):
            print(f"エラー: ファイル '{file_path}' が見つかりません")
            exit(1)
            
        # データの読み込みと前処理
        print(f"ファイル '{file_path}' を読み込んでいます...")
        df = clean_and_prepare_data(file_path)
        
        if df is None or len(df) == 0:
            print("有効なデータがありません")
            exit(1)
            
        print(f"データ読み込み完了: {len(df)}行の有効なデータ")
        print("データサンプル:")
        print(df.head())
        
        # 基本統計量
        print("\n基本統計量:")
        print(df.describe())
        
        # 解析と出力
        a_values = [1.0,1.5,2.0]
        results = calculate_thresholds_and_export_csv(df, output_path, a_values)
        
        print("処理が完了しました。")
    
    except Exception as e:
        print(f"エラー: {e}")
        import traceback
        traceback.print_exc()

ファイル 'G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/data/Tshizuoka2023_0404_50features.csv' を読み込んでいます...
検出された区切り文字: ','
元のカラム: ['InspectionDateAndId', 'MoCA', 'Age']
データ読み込み完了: 733行の有効なデータ
データサンプル:
   MoCA  Age
0    27   66
1    27   69
2    26   77
3    24   79
4    23   71

基本統計量:
             MoCA         Age
count  733.000000  733.000000
mean    24.030014   73.870396
std      3.665177    4.270033
min      9.000000   64.000000
25%     22.000000   70.000000
50%     24.000000   75.000000
75%     27.000000   77.000000
max     30.000000   85.000000
詳細な集計結果をCSVに出力しました: G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/result\moca_detailed_a1.0.csv
シンプルな集計結果をCSVに出力しました: G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/result\moca_simple_a1.0.csv
詳細な集計結果をCSVに出力しました: G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/result\moca_detailed_a1.5.csv
シンプルな集計結果をCSVに出力しました: G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/result\moca_simple_a1.5.csv
詳細な集計結果をCSVに出力しました: G:/共有ドライブ/GAP_長寿研/user/iwamoto/視線の動きの俊敏さ/result\moca_detailed_a2.0